<a href="https://colab.research.google.com/github/lucaskawazoi/lkk-xview2/blob/master/localization/Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.xview2.org/challenge

# Import libraries

In [1]:
from pathlib import Path
import tensorflow as tf
print('TF version:', tf.__version__)

TF version: 1.15.0


# flags main.py

In [0]:
flags = tf.app.flags

In [0]:
flags.DEFINE_string('f', '', 'kernel')

# Cloud TPU Cluster Resolver flags
flags.DEFINE_string(
    "tpu", default=None,
    help="The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")
flags.DEFINE_string(
    "tpu_zone", default=None,
    help="[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")
flags.DEFINE_string(
    "gcp_project", default=None,
    help="[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

# Model specific parameters
flags.DEFINE_string("data_dir", "",
                    "Path to directory containing the MNIST dataset")
flags.DEFINE_string("model_dir", None, "Estimator model_dir")
flags.DEFINE_integer("batch_size", 1024,
                     "Mini-batch size for the training. Note that this "
                     "is the global batch size and not the per-shard batch.")
flags.DEFINE_integer("train_steps", 1000, "Total number of training steps.")
flags.DEFINE_integer("eval_steps", 0,
                     "Total number of evaluation steps. If `0`, evaluation "
                     "after training is skipped.")
flags.DEFINE_float("learning_rate", 0.05, "Learning rate.")

flags.DEFINE_bool("use_tpu", True, "Use TPUs rather than plain CPUs")
flags.DEFINE_bool("enable_predict", True, "Do some predictions at the end")
flags.DEFINE_integer("iterations", 50,
                     "Number of iterations per TPU training loop.")
flags.DEFINE_integer("num_shards", 8, "Number of shards (TPU chips).")

In [4]:
FLAGS = flags.FLAGS
FLAGS

# train_dataset

    train_dataset = dataset.Dataset(
        data_sources=file_pattern,
        reader=tf.TFRecordReader,
        decoder=decoder,
        num_samples=splits_to_sizes[split_name],
        items_to_descriptions=_ITEMS_TO_DESCRIPTIONS,
        ignore_label=ignore_label,
        num_classes=num_classes,
        name=dataset_name,
        multi_label=True)

In [0]:
from tensorflow.contrib import slim as contrib_slim
slim = contrib_slim
dataset = slim.dataset

In [0]:
# Variables
dataset_name = 'xview2'
split_name = 'train'
dataset_dir = 'gs://lkk-xview2/xBD/spacenet_gt/images'

In [0]:
# file_pattern
import os
_FILE_PATTERN = '%s-*'
file_pattern = _FILE_PATTERN
file_pattern = os.path.join(dataset_dir, file_pattern % split_name)
file_pattern

'gs://lkk-xview2/xBD/spacenet_gt/images/train-*'

In [0]:
# decoder
tfexample_decoder = slim.tfexample_decoder

keys_to_features = {
      'image/encoded': tf.FixedLenFeature(
          (), tf.string, default_value=''),
      'image/filename': tf.FixedLenFeature(
          (), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature(
          (), tf.string, default_value='jpeg'),
      'image/height': tf.FixedLenFeature(
          (), tf.int64, default_value=0),
      'image/width': tf.FixedLenFeature(
          (), tf.int64, default_value=0),
      'image/segmentation/class/encoded': tf.FixedLenFeature(
          (), tf.string, default_value=''),
      'image/segmentation/class/format': tf.FixedLenFeature(
          (), tf.string, default_value='png'),
  }

items_to_handlers = {
      'image': tfexample_decoder.Image(
          image_key='image/encoded',
          format_key='image/format',
          channels=3),
      'image_name': tfexample_decoder.Tensor('image/filename'),
      'height': tfexample_decoder.Tensor('image/height'),
      'width': tfexample_decoder.Tensor('image/width'),
      'labels_class': tfexample_decoder.Image(
          image_key='image/segmentation/class/encoded',
          format_key='image/segmentation/class/format',
          channels=1),
  }

decoder = tfexample_decoder.TFExampleDecoder(
      keys_to_features, items_to_handlers)
decoder

In [0]:
# split_to_sizes

import collections

# Named tuple to describe the dataset properties.
DatasetDescriptor = collections.namedtuple(
    'DatasetDescriptor',
    ['splits_to_sizes',   # Splits of the dataset into training, val, and test.
     'num_classes',   # Number of semantic classes, including the background
                      # class (if exists). For example, there are 20
                      # foreground classes + 1 background class in the PASCAL
                      # VOC 2012 dataset. Thus, we set num_classes=21.
     'ignore_label',  # Ignore label value.
    ]
)

_XVIEW2_INFORMATION = DatasetDescriptor(
    splits_to_sizes={
        'train': 1712,
        'val': 571,
    },
    num_classes=2,
    ignore_label=255,
)

_DATASETS_INFORMATION = {
    'xview2': _XVIEW2_INFORMATION
    }

splits_to_sizes = _DATASETS_INFORMATION[dataset_name].splits_to_sizes
splits_to_sizes

{'train': 1712, 'val': 571}

In [0]:
# _ITEMS_TO_DESCRIPTIONS
_ITEMS_TO_DESCRIPTIONS = {
    'image': 'A color image of varying height and width.',
    'labels_class': ('A semantic segmentation label whose size matches image.'
                     'Its values range from 0 (background) to num_classes.'),
}
_ITEMS_TO_DESCRIPTIONS

{'image': 'A color image of varying height and width.',
 'labels_class': 'A semantic segmentation label whose size matches image.Its values range from 0 (background) to num_classes.'}

In [0]:
# ignore_label
ignore_label = _DATASETS_INFORMATION[dataset_name].ignore_label
ignore_label

255

In [0]:
# num_classes
num_classes = _DATASETS_INFORMATION[dataset_name].num_classes
num_classes

2

In [0]:
train_dataset = dataset.Dataset(
    data_sources=file_pattern,
    reader=tf.TFRecordReader,
    decoder=decoder,
    num_samples=splits_to_sizes[split_name],
    items_to_descriptions=_ITEMS_TO_DESCRIPTIONS,
    ignore_label=ignore_label,
    num_classes=num_classes,
    name=dataset_name,
    multi_label=True)
train_dataset

# Eval dataset

In [0]:
# file_pattern
split_name = 'val'
file_pattern = _FILE_PATTERN
file_pattern = os.path.join(dataset_dir, file_pattern % split_name)
file_pattern

'gs://lkk-xview2/xBD/spacenet_gt/images/val-*'

In [0]:
split_name = 'val'
eval_dataset = dataset.Dataset(
    data_sources=file_pattern,
    reader=tf.TFRecordReader,
    decoder=decoder,
    num_samples=splits_to_sizes[split_name],
    items_to_descriptions=_ITEMS_TO_DESCRIPTIONS,
    ignore_label=ignore_label,
    num_classes=num_classes,
    name=dataset_name,
    multi_label=True)

eval_dataset

In [0]:
# batch size
train_batch_size = 64

num_train_images = train_dataset.num_samples
num_classes = train_dataset.num_classes
ignore_label = train_dataset.ignore_label
num_batches_per_epoch = num_train_images / train_batch_size

print(num_train_images, num_classes, ignore_label, num_batches_per_epoch)

1712 2 255 26.75


# Config TPU

In [0]:
# Config credentials to run own TPU
# !gsutil cp gs://[JSON_FILE] gcloud_key_file.json
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gcloud_key_file.json"

In [0]:
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      FLAGS.tpu if (FLAGS.tpu or FLAGS.use_tpu) else "",
      zone=FLAGS.tpu_zone,
      project=FLAGS.gcp_project
  )

In [0]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu="lucas",
    zone='us-central1-f',
    project='lkk-project-1')
tpu_cluster_resolver

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=None,
    save_checkpoints_steps=2000,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=2000,
        num_shards=8))
config

# flags common

In [0]:
flags_common = tf.app.flags

In [0]:
# Flags for input preprocessing.

flags_common.DEFINE_integer('min_resize_value', None,
                     'Desired size of the smaller image side.')

flags_common.DEFINE_integer('max_resize_value', None,
                     'Maximum allowed size of the larger image side.')

flags_common.DEFINE_integer('resize_factor', None,
                     'Resized dimensions are multiple of factor plus one.')

flags_common.DEFINE_boolean('keep_aspect_ratio', True,
                     'Keep aspect ratio after resizing or not.')

# Model dependent flags.

flags_common.DEFINE_integer('logits_kernel_size', 1,
                     'The kernel size for the convolutional kernel that '
                     'generates logits.')

# When using 'mobilent_v2', we set atrous_rates = decoder_output_stride = None.
# When using 'xception_65' or 'resnet_v1' model variants, we set
# atrous_rates = [6, 12, 18] (output stride 16) and decoder_output_stride = 4.
# See core/feature_extractor.py for supported model variants.
flags_common.DEFINE_string('model_variant', 'mobilenet_v2', 'DeepLab model variant.')

flags_common.DEFINE_multi_float('image_pyramid', None,
                         'Input scales for multi-scale feature extraction.')

flags_common.DEFINE_boolean('add_image_level_feature', True,
                     'Add image level feature.')

flags_common.DEFINE_list(
    'image_pooling_crop_size', None,
    'Image pooling crop size [height, width] used in the ASPP module. When '
    'value is None, the model performs image pooling with "crop_size". This'
    'flag is useful when one likes to use different image pooling sizes.')

flags_common.DEFINE_list(
    'image_pooling_stride', '1,1',
    'Image pooling stride [height, width] used in the ASPP image pooling. ')

flags_common.DEFINE_boolean('aspp_with_batch_norm', True,
                     'Use batch norm parameters for ASPP or not.')

flags_common.DEFINE_boolean('aspp_with_separable_conv', True,
                     'Use separable convolution for ASPP or not.')

# Defaults to None. Set multi_grid = [1, 2, 4] when using provided
# 'resnet_v1_{50,101}_beta' checkpoints.
flags_common.DEFINE_multi_integer('multi_grid', None,
                           'Employ a hierarchy of atrous rates for ResNet.')

flags_common.DEFINE_float('depth_multiplier', 1.0,
                   'Multiplier for the depth (number of channels) for all '
                   'convolution ops used in MobileNet.')

flags_common.DEFINE_integer('divisible_by', None,
                     'An integer that ensures the layer # channels are '
                     'divisible by this value. Used in MobileNet.')

# For `xception_65`, use decoder_output_stride = 4. For `mobilenet_v2`, use
# decoder_output_stride = None.
flags_common.DEFINE_list('decoder_output_stride', None,
                  'Comma-separated list of strings with the number specifying '
                  'output stride of low-level features at each network level.'
                  'Current semantic segmentation implementation assumes at '
                  'most one output stride (i.e., either None or a list with '
                  'only one element.')

flags_common.DEFINE_boolean('decoder_use_separable_conv', True,
                     'Employ separable convolution for decoder or not.')

flags_common.DEFINE_enum('merge_method', 'max', ['max', 'avg'],
                  'Scheme to merge multi scale features.')

flags_common.DEFINE_boolean(
    'prediction_with_upsampled_logits', True,
    'When performing prediction, there are two options: (1) bilinear '
    'upsampling the logits followed by softmax, or (2) softmax followed by '
    'bilinear upsampling.')

flags_common.DEFINE_string(
    'dense_prediction_cell_json',
    '',
    'A JSON file that specifies the dense prediction cell.')

flags_common.DEFINE_integer(
    'nas_stem_output_num_conv_filters', 20,
    'Number of filters of the stem output tensor in NAS models.')

flags_common.DEFINE_bool('nas_use_classification_head', False,
                  'Use image classification head for NAS model variants.')

flags_common.DEFINE_bool('nas_remove_os32_stride', False,
                  'Remove the stride in the output stride 32 branch.')

flags_common.DEFINE_bool('use_bounded_activation', False,
                  'Whether or not to use bounded activations. Bounded '
                  'activations better lend themselves to quantized inference.')

flags_common.DEFINE_boolean('aspp_with_concat_projection', True,
                     'ASPP with concat projection.')

flags_common.DEFINE_boolean('aspp_with_squeeze_and_excitation', False,
                     'ASPP with squeeze and excitation.')

flags_common.DEFINE_integer('aspp_convs_filters', 256, 'ASPP convolution filters.')

flags_common.DEFINE_boolean('decoder_use_sum_merge', False,
                     'Decoder uses simply sum merge.')

flags_common.DEFINE_integer('decoder_filters', 256, 'Decoder filters.')

flags_common.DEFINE_boolean('decoder_output_is_logits', False,
                     'Use decoder output as logits or not.')

flags_common.DEFINE_boolean('image_se_uses_qsigmoid', False, 'Use q-sigmoid.')

flags_common.DEFINE_multi_float(
    'label_weights', None,
    'A list of label weights, each element represents the weight for the label '
    'of its index, for example, label_weights = [0.1, 0.5] means the weight '
    'for label 0 is 0.1 and the weight for label 1 is 0.5. If set as None, all '
    'the labels have the same weight 1.0.')

flags_common.DEFINE_float('batch_norm_decay', 0.9997, 'Batchnorm decay.')

In [0]:
FLAGS_COMMON = flags_common.FLAGS

# Common.py

In [0]:
# Constants

# Perform semantic segmentation predictions.
OUTPUT_TYPE = 'semantic'

# Semantic segmentation item names.
LABELS_CLASS = 'labels_class'
IMAGE = 'image'
HEIGHT = 'height'
WIDTH = 'width'
IMAGE_NAME = 'image_name'
LABEL = 'label'
ORIGINAL_IMAGE = 'original_image'

# Test set name.
TEST_SET = 'test'


class ModelOptions(
    collections.namedtuple('ModelOptions', [
        'outputs_to_num_classes',
        'crop_size',
        'atrous_rates',
        'output_stride',
        'preprocessed_images_dtype',
        'merge_method',
        'add_image_level_feature',
        'image_pooling_crop_size',
        'image_pooling_stride',
        'aspp_with_batch_norm',
        'aspp_with_separable_conv',
        'multi_grid',
        'decoder_output_stride',
        'decoder_use_separable_conv',
        'logits_kernel_size',
        'model_variant',
        'depth_multiplier',
        'divisible_by',
        'prediction_with_upsampled_logits',
        'dense_prediction_cell_config',
        'nas_architecture_options',
        'use_bounded_activation',
        'aspp_with_concat_projection',
        'aspp_with_squeeze_and_excitation',
        'aspp_convs_filters',
        'decoder_use_sum_merge',
        'decoder_filters',
        'decoder_output_is_logits',
        'image_se_uses_qsigmoid',
        'label_weights',
        'sync_batch_norm_method',
        'batch_norm_decay',
    ])):
  """Immutable class to hold model options."""

  __slots__ = ()

  def __new__(cls,
              outputs_to_num_classes,
              crop_size=None,
              atrous_rates=None,
              output_stride=8,
              preprocessed_images_dtype=tf.float32):
    """Constructor to set default values.

    Args:
      outputs_to_num_classes: A dictionary from output type to the number of
        classes. For example, for the task of semantic segmentation with 21
        semantic classes, we would have outputs_to_num_classes['semantic'] = 21.
      crop_size: A tuple [crop_height, crop_width].
      atrous_rates: A list of atrous convolution rates for ASPP.
      output_stride: The ratio of input to output spatial resolution.
      preprocessed_images_dtype: The type after the preprocessing function.

    Returns:
      A new ModelOptions instance.
    """
    dense_prediction_cell_config = None
    if FLAGS_COMMON.dense_prediction_cell_json:
      with tf.gfile.Open(FLAGS_COMMON.dense_prediction_cell_json, 'r') as f:
        dense_prediction_cell_config = json.load(f)
    decoder_output_stride = None
    if FLAGS_COMMON.decoder_output_stride:
      decoder_output_stride = [
          int(x) for x in FLAGS_COMMON.decoder_output_stride]
      if sorted(decoder_output_stride, reverse=True) != decoder_output_stride:
        raise ValueError('Decoder output stride need to be sorted in the '
                         'descending order.')
    image_pooling_crop_size = None
    if FLAGS_COMMON.image_pooling_crop_size:
      image_pooling_crop_size = [int(x) for x in FLAGS_COMMON.image_pooling_crop_size]
    image_pooling_stride = [1, 1]
    if FLAGS_COMMON.image_pooling_stride:
      image_pooling_stride = [int(x) for x in FLAGS_COMMON.image_pooling_stride]
    label_weights = FLAGS_COMMON.label_weights
    if label_weights is None:
      label_weights = 1.0
    nas_architecture_options = {
        'nas_stem_output_num_conv_filters': (
            FLAGS_COMMON.nas_stem_output_num_conv_filters),
        'nas_use_classification_head': FLAGS_COMMON.nas_use_classification_head,
        'nas_remove_os32_stride': FLAGS_COMMON.nas_remove_os32_stride,
    }
    return super(ModelOptions, cls).__new__(
        cls, outputs_to_num_classes, crop_size, atrous_rates, output_stride,
        preprocessed_images_dtype,
        FLAGS_COMMON.merge_method,
        FLAGS_COMMON.add_image_level_feature,
        image_pooling_crop_size,
        image_pooling_stride,
        FLAGS_COMMON.aspp_with_batch_norm,
        FLAGS_COMMON.aspp_with_separable_conv,
        FLAGS_COMMON.multi_grid,
        decoder_output_stride,
        FLAGS_COMMON.decoder_use_separable_conv,
        FLAGS_COMMON.logits_kernel_size,
        FLAGS_COMMON.model_variant,
        FLAGS_COMMON.depth_multiplier,
        FLAGS_COMMON.divisible_by,
        FLAGS_COMMON.prediction_with_upsampled_logits,
        dense_prediction_cell_config,
        nas_architecture_options,
        FLAGS_COMMON.use_bounded_activation,
        FLAGS_COMMON.aspp_with_concat_projection,
        FLAGS_COMMON.aspp_with_squeeze_and_excitation,
        FLAGS_COMMON.aspp_convs_filters,
        FLAGS_COMMON.decoder_use_sum_merge,
        FLAGS_COMMON.decoder_filters,
        FLAGS_COMMON.decoder_output_is_logits,
        FLAGS_COMMON.image_se_uses_qsigmoid,
        label_weights,
        'None',
        FLAGS_COMMON.batch_norm_decay)

  def __deepcopy__(self, memo):
    return ModelOptions(copy.deepcopy(self.outputs_to_num_classes),
                        self.crop_size,
                        self.atrous_rates,
                        self.output_stride,
                        self.preprocessed_images_dtype)

# Params

In [0]:
params = {k: FLAGS_MAIN[k].value for k in FLAGS_MAIN}
params

{'add_image_level_feature': True,
 'alsologtostderr': False,
 'aspp_convs_filters': 256,
 'aspp_with_batch_norm': True,
 'aspp_with_concat_projection': True,
 'aspp_with_separable_conv': True,
 'aspp_with_squeeze_and_excitation': False,
 'atrous_rates': [6, 12, 18],
 'batch_norm_decay': 0.9997,
 'crop_size': [513, 513],
 'dataset_dir': None,
 'dataset_name': 'pascal_voc_seg',
 'decoder_filters': 256,
 'decoder_output_is_logits': False,
 'decoder_output_stride': None,
 'decoder_use_separable_conv': True,
 'decoder_use_sum_merge': False,
 'dense_prediction_cell_json': '',
 'depth_multiplier': 1.0,
 'divisible_by': None,
 'eval_batch_size': 8,
 'eval_split': 'val',
 'eval_timeout': None,
 'f': '/root/.local/share/jupyter/runtime/kernel-b76e1e98-9656-4827-b057-839224b8866f.json',
 'fine_tune_batch_norm': True,
 'gcp_project': None,
 'image_pooling_crop_size': None,
 'image_pooling_stride': ['1', '1'],
 'image_pyramid': None,
 'image_se_uses_qsigmoid': False,
 'init_checkpoint': None,
 'ite

In [0]:
# outputs_to_num_classes
outputs_to_num_classes = {OUTPUT_TYPE: num_classes}
outputs_to_num_classes

{'semantic': 2}

In [0]:
# model_options
model_options = ModelOptions(
        outputs_to_num_classes, params['crop_size'], params['atrous_rates'],
        params['output_stride'],
        preprocessed_images_dtype=(tf.bfloat16 if params['use_bfloat16'] else tf.float32))
model_options

ModelOptions(outputs_to_num_classes={'semantic': 2}, crop_size=[513, 513], atrous_rates=[6, 12, 18], output_stride=16, preprocessed_images_dtype=tf.float32, merge_method='max', add_image_level_feature=True, image_pooling_crop_size=None, image_pooling_stride=[1, 1], aspp_with_batch_norm=True, aspp_with_separable_conv=True, multi_grid=None, decoder_output_stride=None, decoder_use_separable_conv=True, logits_kernel_size=1, model_variant='mobilenet_v2', depth_multiplier=1.0, divisible_by=None, prediction_with_upsampled_logits=True, dense_prediction_cell_config=None, nas_architecture_options={'nas_stem_output_num_conv_filters': 20, 'nas_use_classification_head': False, 'nas_remove_os32_stride': False}, use_bounded_activation=False, aspp_with_concat_projection=True, aspp_with_squeeze_and_excitation=False, aspp_convs_filters=256, decoder_use_sum_merge=False, decoder_filters=256, decoder_output_is_logits=False, image_se_uses_qsigmoid=False, label_weights=1.0, sync_batch_norm_method='None', bat

In [0]:
params.update({'ignore_label': ignore_label,
                 'model_options': model_options,
                 'num_batches_per_epoch': num_batches_per_epoch,
                 'num_classes': num_classes,
                 'outputs_to_num_classes': outputs_to_num_classes})
params

{'add_image_level_feature': True,
 'alsologtostderr': False,
 'aspp_convs_filters': 256,
 'aspp_with_batch_norm': True,
 'aspp_with_concat_projection': True,
 'aspp_with_separable_conv': True,
 'aspp_with_squeeze_and_excitation': False,
 'atrous_rates': [6, 12, 18],
 'batch_norm_decay': 0.9997,
 'crop_size': [513, 513],
 'dataset_dir': None,
 'dataset_name': 'pascal_voc_seg',
 'decoder_filters': 256,
 'decoder_output_is_logits': False,
 'decoder_output_stride': None,
 'decoder_use_separable_conv': True,
 'decoder_use_sum_merge': False,
 'dense_prediction_cell_json': '',
 'depth_multiplier': 1.0,
 'divisible_by': None,
 'eval_batch_size': 8,
 'eval_split': 'val',
 'eval_timeout': None,
 'f': '/root/.local/share/jupyter/runtime/kernel-b76e1e98-9656-4827-b057-839224b8866f.json',
 'fine_tune_batch_norm': True,
 'gcp_project': None,
 'ignore_label': 255,
 'image_pooling_crop_size': None,
 'image_pooling_stride': ['1', '1'],
 'image_pyramid': None,
 'image_se_uses_qsigmoid': False,
 'init_ch

# Deeplab estimator

## model.py

In [0]:
# multi_scale_logits
def multi_scale_logits(images,
                       model_options,
                       image_pyramid,
                       weight_decay=0.0001,
                       is_training=False,
                       fine_tune_batch_norm=False,
                       nas_training_hyper_parameters=None):
  """Gets the logits for multi-scale inputs.

  The returned logits are all downsampled (due to max-pooling layers)
  for both training and evaluation.

  Args:
    images: A tensor of size [batch, height, width, channels].
    model_options: A ModelOptions instance to configure models.
    image_pyramid: Input image scales for multi-scale feature extraction.
    weight_decay: The weight decay for model variables.
    is_training: Is training or not.
    fine_tune_batch_norm: Fine-tune the batch norm parameters or not.
    nas_training_hyper_parameters: A dictionary storing hyper-parameters for
      training nas models. Its keys are:
      - `drop_path_keep_prob`: Probability to keep each path in the cell when
        training.
      - `total_training_steps`: Total training steps to help drop path
        probability calculation.

  Returns:
    outputs_to_scales_to_logits: A map of maps from output_type (e.g.,
      semantic prediction) to a dictionary of multi-scale logits names to
      logits. For each output_type, the dictionary has keys which
      correspond to the scales and values which correspond to the logits.
      For example, if `scales` equals [1.0, 1.5], then the keys would
      include 'merged_logits', 'logits_1.00' and 'logits_1.50'.

  Raises:
    ValueError: If model_options doesn't specify crop_size and its
      add_image_level_feature = True, since add_image_level_feature requires
      crop_size information.
  """
  # Setup default values.
  if not image_pyramid:
    image_pyramid = [1.0]
  crop_height = (
      model_options.crop_size[0]
      if model_options.crop_size else tf.shape(images)[1])
  crop_width = (
      model_options.crop_size[1]
      if model_options.crop_size else tf.shape(images)[2])
  if model_options.image_pooling_crop_size:
    image_pooling_crop_height = model_options.image_pooling_crop_size[0]
    image_pooling_crop_width = model_options.image_pooling_crop_size[1]

  # Compute the height, width for the output logits.
  if model_options.decoder_output_stride:
    logits_output_stride = min(model_options.decoder_output_stride)
  else:
    logits_output_stride = model_options.output_stride

  logits_height = scale_dimension(
      crop_height,
      max(1.0, max(image_pyramid)) / logits_output_stride)
  logits_width = scale_dimension(
      crop_width,
      max(1.0, max(image_pyramid)) / logits_output_stride)

  # Compute the logits for each scale in the image pyramid.
  outputs_to_scales_to_logits = {
      k: {}
      for k in model_options.outputs_to_num_classes
  }

  num_channels = images.get_shape().as_list()[-1]

  for image_scale in image_pyramid:
    if image_scale != 1.0:
      scaled_height = scale_dimension(crop_height, image_scale)
      scaled_width = scale_dimension(crop_width, image_scale)
      scaled_crop_size = [scaled_height, scaled_width]
      scaled_images = _resize_bilinear(images, scaled_crop_size, images.dtype)
      if model_options.crop_size:
        scaled_images.set_shape(
            [None, scaled_height, scaled_width, num_channels])
      # Adjust image_pooling_crop_size accordingly.
      scaled_image_pooling_crop_size = None
      if model_options.image_pooling_crop_size:
        scaled_image_pooling_crop_size = [
            scale_dimension(image_pooling_crop_height, image_scale),
            scale_dimension(image_pooling_crop_width, image_scale)]
    else:
      scaled_crop_size = model_options.crop_size
      scaled_images = images
      scaled_image_pooling_crop_size = model_options.image_pooling_crop_size

    updated_options = model_options._replace(
        crop_size=scaled_crop_size,
        image_pooling_crop_size=scaled_image_pooling_crop_size)
    outputs_to_logits = _get_logits(
        scaled_images,
        updated_options,
        weight_decay=weight_decay,
        reuse=tf.AUTO_REUSE,
        is_training=is_training,
        fine_tune_batch_norm=fine_tune_batch_norm,
        nas_training_hyper_parameters=nas_training_hyper_parameters)

    # Resize the logits to have the same dimension before merging.
    for output in sorted(outputs_to_logits):
      outputs_to_logits[output] = _resize_bilinear(
          outputs_to_logits[output], [logits_height, logits_width],
          outputs_to_logits[output].dtype)

    # Return when only one input scale.
    if len(image_pyramid) == 1:
      for output in sorted(model_options.outputs_to_num_classes):
        outputs_to_scales_to_logits[output][
            MERGED_LOGITS_SCOPE] = outputs_to_logits[output]
      return outputs_to_scales_to_logits

    # Save logits to the output map.
    for output in sorted(model_options.outputs_to_num_classes):
      outputs_to_scales_to_logits[output][
          'logits_%.2f' % image_scale] = outputs_to_logits[output]

  # Merge the logits from all the multi-scale inputs.
  for output in sorted(model_options.outputs_to_num_classes):
    # Concatenate the multi-scale logits for each output type.
    all_logits = [
        tf.expand_dims(logits, axis=4)
        for logits in outputs_to_scales_to_logits[output].values()
    ]
    all_logits = tf.concat(all_logits, 4)
    merge_fn = (
        tf.reduce_max
        if model_options.merge_method == 'max' else tf.reduce_mean)
    outputs_to_scales_to_logits[output][MERGED_LOGITS_SCOPE] = merge_fn(
        all_logits, axis=4)

  return outputs_to_scales_to_logits

In [0]:
# _build_network
def _build_network(features, mode, params):
  """Builds the network for different values of params['use_bfloat16']."""
  if params['use_bfloat16']:
    with bfloat16.bfloat16_scope():
      outputs_to_scales_to_logits = multi_scale_logits(
          features,
          params['model_options'],
          params['image_pyramid'],
          weight_decay=0.0,
          is_training=mode == tf.estimator.ModeKeys.TRAIN,
          fine_tune_batch_norm=(
              params['fine_tune_batch_norm']
              if mode == tf.estimator.ModeKeys.TRAIN else False)
      )
    for level, output in outputs_to_scales_to_logits.iteritems():
      for scale, logits in output.iteritems():
        outputs_to_scales_to_logits[level][scale] = tf.cast(logits, tf.float32)
  else:
    outputs_to_scales_to_logits = multi_scale_logits(
        features,
        params['model_options'],
        params['image_pyramid'],
        weight_decay=params['weight_decay'],
        is_training=mode == tf.estimator.ModeKeys.TRAIN,
        fine_tune_batch_norm=(
            params['fine_tune_batch_norm']
            if mode == tf.estimator.ModeKeys.TRAIN else False)
    )
  return outputs_to_scales_to_logits

In [0]:
# loss_fn
def loss_fn(features, labels, mode, params):
  """Computes label predictions and cross entropy loss against labels."""
  outputs_to_scales_to_logits = _build_network(features, mode, params)

  for output, num_classes in params['outputs_to_num_classes'].items():
    add_softmax_cross_entropy_loss_for_each_scale(
        outputs_to_scales_to_logits[output],
        labels,
        num_classes,
        ignore_label=params['ignore_label'],
        loss_weight=1.0,
        upsample_logits=params['upsample_logits'],
        scope=output)

  losses = tf.add_n(tf.losses.get_losses())
  l2_loss = []
  for v in tf.trainable_variables():
    if 'BatchNorm' not in v.name and 'weights' in v.name:
      l2_loss.append(tf.nn.l2_loss(v))
  loss = losses + params['weight_decay'] * tf.add_n(l2_loss)
  return loss

In [0]:
# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Provide model_fn for TPUEstimator training and evaluation."""

import tensorflow as tf

from tensorflow.contrib.tpu.python.tpu import bfloat16
from deeplab import common
from deeplab.core import feature_extractor
from deeplab.model import multi_scale_logits
from deeplab.utils.train_utils import add_softmax_cross_entropy_loss_for_each_scale


slim = tf.contrib.slim

# Scope for the merged multi-scale logits.
_MERGED_LOGITS_SCOPE = 'merged_logits'




def _create_eval_metric(features, labels, params):
  """Creates eval_metric for model_fn."""
  outputs_to_scales_to_logits = _build_network(
      features, tf.estimator.ModeKeys.EVAL, params)

  semantic_merged_logits = (
      outputs_to_scales_to_logits[common.OUTPUT_TYPE][_MERGED_LOGITS_SCOPE])

  def metric_fn(semantic_merged_logits, labels):
    """Creates metric_fn for TPUEstimatorSpec."""
    logits = tf.image.resize_bilinear(
        semantic_merged_logits, params['crop_size'], align_corners=True)
    predictions_with_shape = tf.argmax(logits, 3, output_type=tf.int32)
    predictions = tf.reshape(predictions_with_shape, shape=[-1])

    labels = tf.reshape(labels, shape=[-1])
    weights = tf.to_float(tf.not_equal(labels, params['ignore_label']))

    # Set ignore_label regions to label 0, because metrics.mean_iou requires
    # range of labels = [0, dataset.num_classes). Note the ignore_lable regions
    # are not evaluated since the corresponding regions contain weights = 0.
    labels = tf.where(
        tf.equal(labels, params['ignore_label']), tf.zeros_like(labels), labels)

    return {
        'miou':
            tf.metrics.mean_iou(
                predictions, labels, params['num_classes'], weights=weights),
    }

  return metric_fn, [semantic_merged_logits, labels]


def _get_optimizer(params, learning_rate):
  """Gets optimizer based on params."""
  if params['optimizer'] == 'momentum':
    optimizer = tf.train.MomentumOptimizer(
        learning_rate=learning_rate,
        momentum=params['momentum'],
        use_nesterov=True)
  elif params['optimizer'] == 'sgd':
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  elif params['optimizer'] == 'rmsprop':
    optimizer = tf.train.RMSPropOptimizer(
        learning_rate=learning_rate,
        epsilon=params['rmsprop_epsilon'],
        momentum=params['rmsprop_momentum'])
  else:
    raise KeyError('Unknown optimizer: %s' % params['optimizer'])

  return optimizer


def _get_learning_rate(params, global_step, num_batches_per_epoch):
  """Gets learning rate based on params."""
  learning_policy = params['learning_policy']
  if learning_policy == 'poly':
    learning_rate = tf.train.polynomial_decay(
        params['learning_rate'],
        global_step,
        params['train_steps'],
        end_learning_rate=0,
        power=params['learning_power'])
  elif learning_policy == 'step':
    learning_rate = tf.train.exponential_decay(
        params['learning_rate'],
        global_step,
        decay_rate=params['learning_rate_decay'],
        decay_steps=num_batches_per_epoch,
        staircase=True,
    )
  else:
    raise KeyError('Unknown learning policy: %s' % learning_policy)

  return learning_rate

In [0]:
def model_fn(features, labels, mode, params):
  """TPUEstimator compatible model function."""
  loss = loss_fn(features, labels, mode, params)

  host_call = None
  train_op = None
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_batches_per_epoch = params['num_batches_per_epoch']
    global_step = tf.train.get_global_step()
    current_epoch = tf.cast(global_step, tf.float32) / num_batches_per_epoch

    learning_rate = _get_learning_rate(
        params, global_step, num_batches_per_epoch)
    optimizer = _get_optimizer(params, learning_rate)
    if params['use_tpu']:
      optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

    # Batch norm requires update_ops to be added as a train_op dependency.
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
      train_op = optimizer.minimize(loss, tf.train.get_global_step())

    if params['use_host_call']:
      def host_call_fn(global_step, loss, learning_rate, current_epoch):
        """Training host call. Creates scalar summaries for training metrics.

        This function is executed on the CPU and should not directly reference
        any Tensors in the rest of the `model_fn`. To pass Tensors from the
        model to the `metric_fn`, provide as part of the `host_call`. See
        https://www.tensorflow.org/api_docs/python/tf/contrib/tpu/TPUEstimatorSpec
        for more information.

        Arguments should match the list of `Tensor` objects passed as the second
        element in the tuple passed to `host_call`.

        Args:
          global_step: `Tensor with shape `[batch, ]` for the global_step.
          loss: `Tensor` with shape `[batch, ]` for the training loss.
          learning_rate: `Tensor` with shape `[batch, ]` for the learning_rate.
          current_epoch: `Tensor` with shape `[batch, ]` for the current_epoch.

        Returns:
          List of summary ops to run on the CPU host.
        """
        # Outfeed supports int32 but global_step is expected to be int64.
        global_step = tf.reduce_mean(global_step)
        with (tf.contrib.summary.create_file_writer(
            params['model_dir']).as_default()):
          with tf.contrib.summary.always_record_summaries():
            tf.contrib.summary.scalar(
                'loss', tf.reduce_mean(loss), step=global_step)
            tf.contrib.summary.scalar(
                'learning_rate', tf.reduce_mean(learning_rate),
                step=global_step)
            tf.contrib.summary.scalar(
                'current_epoch', tf.reduce_mean(current_epoch),
                step=global_step)

            return tf.contrib.summary.all_summary_ops()

      # To log the loss, current learning rate, and epoch for Tensorboard, the
      # summary op needs to be run on the host CPU via host_call. host_call
      # expects [batch_size, ...] Tensors, thus reshape to introduce a batch
      # dimension. These Tensors are implicitly concatenated to
      # [params['batch_size']].
      global_step_t = tf.reshape(global_step, [1])
      loss_t = tf.reshape(loss, [1])
      learning_rate_t = tf.reshape(learning_rate, [1])
      current_epoch_t = tf.reshape(current_epoch, [1])

      host_call = (host_call_fn,
                   [global_step_t, loss_t, learning_rate_t, current_epoch_t])

  eval_metrics = None
  if mode == tf.estimator.ModeKeys.EVAL:
    eval_metrics = _create_eval_metric(features, labels, params)

  # Restore from checkpoint if available.
  if params['init_checkpoint'] and mode == tf.estimator.ModeKeys.TRAIN:
    tf.logging.info('Found an init checkpoint.')
    model_variant = params['model_options'].model_variant
    var_scope = '{}/'.format(feature_extractor.name_scope[model_variant])
    def scaffold_fn():
      """Create Scaffold for initialization, etc."""
      tf.train.init_from_checkpoint(params['init_checkpoint'], {
          var_scope: var_scope,
      })
      return tf.train.Scaffold()
  else:
    tf.logging.info('No init checkpoint found. Training from scratch.')
    scaffold_fn = None
  return tf.contrib.tpu.TPUEstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
      scaffold_fn=scaffold_fn,
      host_call=host_call,
      eval_metrics=eval_metrics,
  )


## deeplab_estimator

In [0]:
deeplab_estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=FLAGS_MAIN.use_tpu,
      model_fn=model.model_fn,
      config=config,
      train_batch_size=FLAGS_MAIN.train_batch_size,
      eval_batch_size=FLAGS_MAIN.eval_batch_size,
      params=params)
deeplab_estimator

NameError: ignored